In [2]:
import json
import math

from numpy.random import choice, seed
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

from models import BasicTransformer

## Data Preparation

In [3]:
import pandas as pd
from preprocess_data import *

with open('action_types.json', 'r') as f:
    action_types = json.load(f)
    
df = (
    pd.read_csv("WSL_actions.csv", index_col = 0)
    .pipe(add_coordinate_bins, n_bins_x = 10, n_bins_y = 10)
    .pipe(add_team_as_dummy)
    .pipe(get_action_type_names, action_types)
    .pipe(get_action_tokens)
    .assign(
        group_id = lambda d: d.groupby(['game_id', 'period_id']).ngroup(),
        action_token = lambda d: pd.Categorical(d.action_token)
    )
    [['group_id', 'action_token']]
)

vocab = df['action_token'].cat.categories

In [24]:
seed(42)
train_groups = choice(df['group_id'].unique(), int(0.8 * df['group_id'].nunique()), replace = False)
val_groups = choice(train_groups, int(0.8 * len(train_groups)), replace = False)
train_groups[:5]

array([ 55, 363, 406, 428, 402], dtype=int64)

In [25]:
train_df = df.query("group_id.isin(@train_groups) and ~group_id.isin(@val_groups)")
val_df = df.query("group_id.isin(@val_groups)")

X_train = np.lib.stride_tricks.sliding_window_view(train_df['action_token'].map(list(vocab).index), (3,))[:-1]
y_train = np.lib.stride_tricks.sliding_window_view(train_df['action_token'].map(list(vocab).index), (3,))[1:]

X_val = np.lib.stride_tricks.sliding_window_view(val_df['action_token'].map(list(vocab).index), (3,))[:-1]
y_val = np.lib.stride_tricks.sliding_window_view(val_df['action_token'].map(list(vocab).index), (3,))[1:]

train_loader = torch.utils.data.DataLoader(
    list(zip(X_train, y_train)),
    batch_size=32,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    list(zip(X_val, y_val)),
    batch_size=32,
    shuffle=True
)

In [26]:
def accuracy(preds, labels):
    return sum(preds[:, -1].argmax(dim=1) == labels[:, -1].argmax(dim=1)) / len(preds)

## Training Loop

In [27]:
def train(model, n_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(n_epochs):
        model.train()
        i = 0
        train_loss = 0
        train_acc = 0
        for x_bat, y_bat in iter(train_loader):
            x_bat.permute(1, 0)
            y_bat = F.one_hot(y_bat, num_classes=len(vocab)).float()
            optimizer.zero_grad()
            y_pred = model(x_bat)
            loss = criterion(y_pred, y_bat)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            i += 1
            train_acc += accuracy(y_pred, y_bat)
        
            # print(f'Epoch {epoch}, iter {i}, loss: {loss.item()}')
        train_acc = train_acc / len(train_loader)

        val_loss = 0
        val_acc = 0
        for x_val, y_val in iter(val_loader):
            y_pred = model(x_val)
            y_val = F.one_hot(y_val, num_classes=len(vocab)).float()
            loss = criterion(y_pred, y_val)
            val_loss += loss.item()
            val_acc += accuracy(y_pred, y_val)
        
        val_acc = val_acc / len(val_loader)
        print(f'Epoch {epoch}, iter {i}, train_loss: {train_loss}, train_acc: {train_acc}, val_loss: {val_loss}, val_acc: {val_acc}')

In [28]:
model = BasicTransformer(len(vocab), 128, 128, 2, 2, 0.1)

In [29]:
train(model)

c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py:171: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:212.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


In [ ]:
torch.save(model.state_dict(), "transformer_weights.pkl")